# Close the Budgets

In [1]:
import fastjmd95 # documentation here: https://github.com/xgcm/fastjmd95
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import os
from xmitgcm import open_mdsdataset
import xgcm


from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import ecco_masks
from analysis_package import integrate_zonally
from analysis_package import lat_fields
from analysis_package import convert_to_sigspace
from analysis_package import calculate_isovol

import sys
sys.path.append('/Users/tatsumonkman/ECCOv4-py/ECCOv4-py')
import ecco_v4_py as ecco


from analysis_package import plotting_functions
from analysis_package import open_datasets

open_datasets = reload(open_datasets)
convert_to_sigspace = reload(convert_to_sigspace)

load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary

In [2]:
######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################
grid_path = "./ecco_grid/ECCOv4r3_grid.nc"
grid = xr.open_dataset(grid_path)
cds = grid.coords.to_dataset()
grid_xmitgcm = ecco.ecco_utils.get_llc_grid(cds)

######################################################################################################################
############################################# CREATE DOMAIN MASKS ####################################################
######################################################################################################################
maskW = xr.open_dataarray("generic_masks/maskW.nc")
maskS = xr.open_dataarray("generic_masks/maskS.nc")
maskC = xr.open_dataarray("generic_masks/maskC.nc")

southern_ocean_mask_W, southern_ocean_mask_S, southern_ocean_mask_C, so_atl_basin_mask_W, so_atl_basin_mask_S, so_atl_basin_mask_C, so_indpac_basin_mask_W, so_indpac_basin_mask_S, so_indpac_basin_mask_C = ecco_masks.get_basin_masks(maskW, maskS, maskC)

baffin_mask_C = ecco.get_basin_mask("baffin",maskC)
north_mask_C = ecco.get_basin_mask("north",maskC)
hudson_mask_C = ecco.get_basin_mask("hudson",maskC)
gin_mask_C = ecco.get_basin_mask("gin",maskC)
bering_mask_C = ecco.get_basin_mask("bering",maskC)
okhotsk_mask_C = ecco.get_basin_mask("okhotsk",maskC)
atl_mask_C = ecco.get_basin_mask("atl",maskC)

maskC_low_depth_filtered = maskC*maskC.where(grid.Depth > 50.)

# masks with low depths filtered out
so_atl_maskC_low_depth_filtered = so_atl_basin_mask_C*maskC.where(grid.Depth > 100.)
so_indpac_maskC_low_depth_filtered = so_indpac_basin_mask_C*maskC.where(grid.Depth > 100.)




mexico_maskC = ecco.get_basin_mask("mexico",maskC)
mexico_maskC = mexico_maskC.where(mexico_maskC!=0,other=np.nan)
#plotting_functions.world_plot(mexico_maskC)
#plotting_functions.world_plot(so_atl_basin_mask_C.where(mexico_maskC!=1))
# Atlantic mask no caribbean
so_atl_nomex_basin_mask_C = so_atl_basin_mask_C.where(mexico_maskC!=1)

atl_midlat_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=50,other=np.nan).where(so_atl_basin_mask_C.lat>=-32,other=np.nan)
indpac_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat>=-32,other=np.nan)
so_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat<-32,other=np.nan)


ecco_grid = grid.copy()


load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary

In [ ]:
SO_sig2_mask = xr.open_dataarray("SO_sig2_mask_highres.nc")
ATL_sig2_mask = xr.open_dataarray("ATL_sig2_mask_highres.nc")



# Load data

In [3]:
SALT_yearly = xr.open_dataarray("SALT_yearly_mean.nc")
THETA_yearly = xr.open_dataarray("THETA_yearly_mean.nc")

IPMOC_sig2 = xr.open_dataarray("IPMOC_sig2.nc")
AMOC_sig2 = xr.open_dataarray("AMOC_sig2.nc")

ds_llc = open_mdsdataset('global_oce_llc90/', iters=8, geometry='llc')

# Volume (m^3)
vol = (ds_llc.rA*ds_llc.drF*ds_llc.hFacC).transpose('face','k','j','i')
vol = vol.rename({"face":"tile"})


calc_drhodx = False
if calc_drhodx:
    drhodt = fastjmd95.drhodt(SALT_yearly, THETA_yearly, 2000)
    drhods = fastjmd95.drhods(SALT_yearly, THETA_yearly, 2000)
    drhodt.to_netcdf("drhodt.nc")
    drhods.to_netcdf("drhods.nc")
else:
    drhodt = xr.open_dataarray("drhodt.nc")
    drhods = xr.open_dataarray("drhods.nc")

In [4]:
calc_ETAN_yearly = False


if calc_ETAN_yearly:
    
    ETAN = xr.open_dataset("./ETAN_snaps_full.nc")
    ETAN_yearly = SALT_yearly.isel(k=0).copy()

    for t in range(1,25):
        ETAN_yearly.loc[{"time":t}] = ETAN.ETAN.isel(time=np.arange(t*12,t*12+12)).mean(dim="time")

    ETAN_yearly.loc[{"time":0}] = ETAN.ETAN.isel(time=np.arange(2,12)).mean(dim="time")
    ETAN_yearly.loc[{"time":25}] = ETAN.ETAN.isel(time=np.arange(t*12,t*12+13)).mean(dim="time")

    ETAN_yearly.assign_coords({"time":np.arange(0,26)})
    ETAN_yearly.name = "ETAN"
    ETAN_yearly.to_netcdf("./ETAN_yearly.nc")
    
else:
    ETAN_yearly = xr.open_dataarray("./ETAN_yearly.nc").assign_coords({"time":np.arange(0,26)})

# Download / make yearly mean files

In [5]:
calc_yearly = False

variables = [
             "ADVr_SLT",
             "ADVr_TH",
    
             "ADVx_SLT",
             "ADVx_TH",
    
             "ADVy_SLT",
             "ADVy_TH",
    
             "DFrE_SLT",
             "DFrE_TH",
    
             "DFrI_SLT",
             "DFrI_TH",
    
             "DFxE_SLT",
             "DFxE_TH",    
    
             "DFyE_SLT",
             "DFyE_TH",    
                                    
            ]


yearly_mean_transport_terms = []
if calc_yearly:
    for var in variables:
        print(var,end=" ")
        tmp_var = xr.open_dataset(f"./{var}_full.nc")[var]
        tmp_var_yearly = tmp_var.isel(time=np.arange(0,26))
        for time in np.arange(0,312,12):
            print(time,end=" ")
            tmp_var_yearly.loc[{"time":time/12}] = tmp_var.isel(time=np.arange(time,time+12)).mean(dim="time")
        tmp_var_yearly.to_netcdf(f"./{var}_yearly.nc")
        yearly_mean_transport_terms.append(tmp_var_yearly)
        tmp_var.close()
        tmp_var_yearly.close()

        
else:
    for var in variables:
        print(var,end=" ")
        # divide by volume to get a vertical tendency..
        tmp_var_yearly = xr.open_dataset(f"./{var}_yearly.nc")
        yearly_mean_transport_terms.append(tmp_var_yearly)    
        tmp_var_yearly.close()

ADVr_SLT ADVr_TH ADVx_SLT ADVx_TH ADVy_SLT ADVy_TH DFrE_SLT DFrE_TH DFrI_SLT DFrI_TH DFxE_SLT DFxE_TH DFyE_SLT DFyE_TH 

In [6]:
xr_yearly_mean_transport_terms = xr.merge(yearly_mean_transport_terms)

In [7]:
SO_sig_mask = xr.open_dataarray("SO_sig_mask.nc")

# Define sigma values

In [8]:


sigma2_lvls = np.arange(1036.0,1037.24,0.02)

SIGMA2_yearly = xr.open_dataarray("SIGMA2_yearly.nc")


# Calculate vertical flux across sig2 boundary
Just to start, see if vertical flux gives us something that looks like our expected transport

In [9]:
recalc_tendencies = False

if recalc_tendencies:
    grid = ecco.get_llc_grid(ecco_grid)

    #############
    # Advective heat and salt convergence
    #############
    # diff 2d vector aha what does this do?
    ADVxy_TH_diff = grid.diff_2d_vector({'X' : xr_yearly_mean_transport_terms.ADVx_TH, 'Y' : xr_yearly_mean_transport_terms.ADVy_TH}, boundary = 'fill')
    print("chkpt 0")
    
    # Convergence of horizontal advection (degC m^3/s)
    adv_TH_hConvH = (-(ADVxy_TH_diff['X'] + ADVxy_TH_diff['Y']))
    print("chkpt 1")
    
    # Load monthly averages of vertical advective fluxes
    ADVr_TH = xr_yearly_mean_transport_terms.ADVr_TH.transpose('time','tile','k_l','j','i')
    print("chkpt 2")
    
    # Convergence of vertical advection (degC m^3/s)
    adv_TH_vConvH = grid.diff(ADVr_TH, 'Z', boundary='fill')
    print("chkpt 3")
    
    # Sum horizontal and vertical convergences and divide by volume (degC/s)
    G_TH_advection = (adv_TH_hConvH + adv_TH_vConvH)/vol
    print("chkpt 4")
    
    # diff 2d vector aha what does this do?
    ADVxy_SLT_diff = grid.diff_2d_vector({'X' : xr_yearly_mean_transport_terms.ADVx_SLT, 'Y' : xr_yearly_mean_transport_terms.ADVy_SLT}, boundary = 'fill')
    print("chkpt 5")
    
    # Convergence of horizontal advection (psu m^3/s)
    adv_SLT_hConvH = (-(ADVxy_SLT_diff['X'] + ADVxy_SLT_diff['Y']))
    print("chkpt 6")
    
    # Load monthly averages of vertical advective fluxes
    ADVr_SLT = xr_yearly_mean_transport_terms.ADVr_SLT.transpose('time','tile','k_l','j','i')
    print("chkpt 7")
    
    # Convergence of vertical advection (psu m^3/s)
    adv_SLT_vConvH = grid.diff(ADVr_SLT, 'Z', boundary='fill')
    print("chkpt 8")
    
    # Sum horizontal and vertical convergences and divide by volume (psu/s)
    G_SLT_advection = (adv_SLT_hConvH + adv_SLT_vConvH)/vol
    print("chkpt 9")
    
    #############
    #############




    #############
    # Diffusive heat and salt convergence
    #############
    # diff 2d TRUST IN IAN FENTY
    DFxyE_TH_diff = grid.diff_2d_vector({'X' : xr_yearly_mean_transport_terms.DFxE_TH, 'Y' : xr_yearly_mean_transport_terms.DFyE_TH}, boundary = 'fill')
    print("chkpt 10")
    
    # Convergence of horizontal diffusion (degC m^3/s)
    dif_TH_hConvH = (-(DFxyE_TH_diff['X'] + DFxyE_TH_diff['Y']))
    print("chkpt 11")
    
    # Load monthly averages of vertical diffusive fluxes
    DFrE_TH = xr_yearly_mean_transport_terms.DFrE_TH.transpose('time','tile','k_l','j','i')
    DFrI_TH = xr_yearly_mean_transport_terms.DFrI_TH.transpose('time','tile','k_l','j','i')
    print("chkpt 12")
    
    # Convergence of vertical diffusion (degC m^3/s)
    dif_TH_vConvH = grid.diff(DFrE_TH, 'Z', boundary='fill') + grid.diff(DFrI_TH, 'Z', boundary='fill')
    print("chkpt 13")
    
    # Sum horizontal and vertical convergences and divide by volume (degC/s)
    G_TH_diffusion = (dif_TH_hConvH + dif_TH_vConvH)/vol
    print("chkpt 14")
    
    # diff 2d TRUST IN IAN FENTY
    DFxyE_SLT_diff = grid.diff_2d_vector({'X' : xr_yearly_mean_transport_terms.DFxE_SLT, 'Y' : xr_yearly_mean_transport_terms.DFyE_SLT}, boundary = 'fill')
    print("chkpt 15")
    
    # Convergence of horizontal diffusion (psu m^3/s)
    dif_SLT_hConvH = (-(DFxyE_SLT_diff['X'] + DFxyE_SLT_diff['Y']))
    print("chkpt 16")
    
    # Load monthly averages of vertical diffusive fluxes
    DFrE_SLT = xr_yearly_mean_transport_terms.DFrE_SLT.transpose('time','tile','k_l','j','i')
    DFrI_SLT = xr_yearly_mean_transport_terms.DFrI_SLT.transpose('time','tile','k_l','j','i')
    print("chkpt 17")
    
    # Convergence of vertical diffusion (psu m^3/s)
    dif_SLT_vConvH = grid.diff(DFrE_SLT, 'Z', boundary='fill') + grid.diff(DFrI_SLT, 'Z', boundary='fill')
    print("chkpt 18")
    
    # Sum horizontal and vertical convergences and divide by volume (psu/s)
    G_SLT_diffusion = (dif_SLT_hConvH + dif_SLT_vConvH)/vol
    #############
    #############
    
    prefix = "./Small_outputs/"
    suffix = "_yearly.nc"
    
    adv_TH_vConvH.to_netcdf(f"{prefix}adv_TH_vConvH{suffix}")
    adv_SLT_vConvH.to_netcdf(f"{prefix}adv_SLT_vConvH{suffix}")
    dif_TH_vConvH.to_netcdf(f"{prefix}dif_TH_vConvH{suffix}")
    dif_SLT_vConvH.to_netcdf(f"{prefix}dif_SLT_vConvH{suffix}")
    adv_TH_hConvH.to_netcdf(f"{prefix}adv_TH_hConvH{suffix}")
    adv_SLT_hConvH.to_netcdf(f"{prefix}adv_SLT_hConvH{suffix}")
    dif_TH_hConvH.to_netcdf(f"{prefix}dif_TH_hConvH{suffix}")
    dif_SLT_hConvH.to_netcdf(f"{prefix}dif_SLT_hConvH{suffix}")
                             
else:
    
    prefix = "./Small_outputs/"
    suffix = "_yearly.nc"
    
    adv_TH_vConvH = xr.open_dataarray(f"{prefix}adv_TH_vConvH{suffix}")
    adv_SLT_vConvH = xr.open_dataarray(f"{prefix}adv_SLT_vConvH{suffix}")
    dif_TH_vConvH = xr.open_dataarray(f"{prefix}dif_TH_vConvH{suffix}")
    dif_SLT_vConvH = xr.open_dataarray(f"{prefix}dif_SLT_vConvH{suffix}")
    adv_TH_hConvH = xr.open_dataarray(f"{prefix}adv_TH_hConvH{suffix}")
    adv_SLT_hConvH = xr.open_dataarray(f"{prefix}adv_SLT_hConvH{suffix}")
    dif_TH_hConvH = xr.open_dataarray(f"{prefix}dif_TH_hConvH{suffix}")
    dif_SLT_hConvH = xr.open_dataarray(f"{prefix}dif_SLT_hConvH{suffix}")
                            

In [11]:
calc_nointerp = False
load_nointerp = False
# calculate 
prefix = "./"

if calc_nointerp:
    grid_nans = ecco_grid.hFacC.where(ecco_grid.hFacC!=0,other=np.nan)
    
    # calculate EOF coefficients
    drhodt = fastjmd95.drhodt(SALT_yearly, THETA_yearly, 2000)
    drhods = fastjmd95.drhods(SALT_yearly, THETA_yearly, 2000)

    sigma2_lvls = np.arange(1036.8,1037.24,0.02)

    


    T_TH_vertical_advection = convert_to_sigspace.integrate_vertically_in_sig2space(drhodt*adv_TH_vConvH,
                                                                 SIGMA2_yearly,
                                                                 sigma2_lvls,ecco_grid,interpolate=False)

    T_SLT_vertical_advection = convert_to_sigspace.integrate_vertically_in_sig2space(drhods*adv_SLT_vConvH,
                                                                                     SIGMA2_yearly,
                                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_TH_vertical_diffusion = convert_to_sigspace.integrate_vertically_in_sig2space(drhodt*dif_TH_vConvH,
                                                                     SIGMA2_yearly,
                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_SLT_vertical_diffusion = convert_to_sigspace.integrate_vertically_in_sig2space(drhods*dif_SLT_vConvH,
                                                                     SIGMA2_yearly,
                                                                     sigma2_lvls,ecco_grid,interpolate=False)
    T_TH_vertical_advection.to_netcdf(f"{prefix}T_TH_vertical_advection_yearly.nc")
    T_SLT_vertical_advection.to_netcdf(f"{prefix}T_SLT_vertical_advection_yeary.nc")
    T_TH_vertical_diffusion.to_netcdf(f"{prefix}T_TH_vertical_diffusion_yearly.nc")
    T_SLT_vertical_diffusion.to_netcdf(f"{prefix}T_SLT_vertical_diffusion_yearly.nc")

    
    T_TH_horizontal_advection = convert_to_sigspace.integrate_vertically_in_sig2space(drhodt*adv_TH_hConvH,
                                                                     SIGMA2_yearly,
                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_SLT_horizontal_advection = convert_to_sigspace.integrate_vertically_in_sig2space(drhods*adv_SLT_hConvH,
                                                                                     SIGMA2_yearly,
                                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_TH_horizontal_diffusion = convert_to_sigspace.integrate_vertically_in_sig2space(drhodt*dif_TH_hConvH,
                                                                     SIGMA2_yearly,
                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_SLT_horizontal_diffusion = convert_to_sigspace.integrate_vertically_in_sig2space(drhods*dif_SLT_hConvH,
                                                                     SIGMA2_yearly,
                                                                     sigma2_lvls,ecco_grid,interpolate=False)

    T_TH_horizontal_advection.to_netcdf(f"{prefix}T_TH_vertical_advection_yearly.nc")
    T_SLT_horizontal_advection.to_netcdf(f"{prefix}T_SLT_vertical_advection_yeary.nc")
    T_TH_horizontal_diffusion.to_netcdf(f"{prefix}T_TH_vertical_diffusion_yearly.nc")
    T_SLT_horizontal_diffusion.to_netcdf(f"{prefix}T_SLT_vertical_diffusion_yearly.nc")

   

    
if load_nointerp:
    T_TH_horizontal_advection = xr.open_dataarray(f"{prefix}T_TH_horizontal_advection_yearly.nc").differentiate("sig").mean(dim="time")
    T_SLT_horizontal_advection = xr.open_dataarray(f"{prefix}T_SLT_horizontal_advection_yearly.nc").differentiate("sig").mean(dim="time")
    T_TH_horizontal_diffusion = xr.open_dataarray(f"{prefix}T_TH_horizontal_diffusion_yearly.nc").differentiate("sig").mean(dim="time")
    T_SLT_horizontal_diffusion = xr.open_dataarray(f"{prefix}T_SLT_horizontal_diffusion_yearly.nc").differentiate("sig").mean(dim="time")
    T_TH_vertical_advection = xr.open_dataarray(f"{prefix}T_TH_vertical_advection_yearly.nc").differentiate("sig").mean(dim="time")
    T_SLT_vertical_advection = xr.open_dataarray(f"{prefix}T_SLT_vertical_advection_yeary.nc").differentiate("sig").mean(dim="time")
    T_TH_vertical_diffusion = xr.open_dataarray(f"{prefix}T_TH_vertical_diffusion_yearly.nc").differentiate("sig").mean(dim="time")
    T_SLT_vertical_diffusion = xr.open_dataarray(f"{prefix}T_SLT_vertical_diffusion_yearly.nc").differentiate("sig").mean(dim="time") 
    

In [13]:
SO_sig2_mask = xr.open_dataarray("SO_sig2_mask_highres.nc")
ATL_sig2_mask = xr.open_dataarray("ATL_sig2_mask_highres.nc")



In [15]:

    
ip_T_TH_vertical_advection = T_TH_vertical_advection*indpac_basin_mask_C
ip_T_SLT_vertical_advection = T_SLT_vertical_advection*indpac_basin_mask_C
ip_T_TH_vertical_diffusion = T_TH_vertical_diffusion*indpac_basin_mask_C
ip_T_SLT_vertical_diffusion = T_SLT_vertical_diffusion*indpac_basin_mask_C

ip_T_TH_horizontal_advection = T_TH_horizontal_advection*indpac_basin_mask_C
ip_T_SLT_horizontal_advection = T_SLT_horizontal_advection*indpac_basin_mask_C
ip_T_TH_horizontal_diffusion = T_TH_horizontal_diffusion*indpac_basin_mask_C
ip_T_SLT_horizontal_diffusion = T_SLT_horizontal_diffusion*indpac_basin_mask_C

ip_T_forcS = T_forcS*indpac_basin_mask_C




atl_T_TH_vertical_advection = T_TH_vertical_advection*ATL_sig2_mask
atl_T_SLT_vertical_advection = T_SLT_vertical_advection*ATL_sig2_mask
atl_T_TH_vertical_diffusion = T_TH_vertical_diffusion*ATL_sig2_mask
atl_T_SLT_vertical_diffusion = T_SLT_vertical_diffusion*ATL_sig2_mask

atl_T_TH_horizontal_advection = T_TH_horizontal_advection*ATL_sig2_mask
atl_T_SLT_horizontal_advection = T_SLT_horizontal_advection*ATL_sig2_mask
atl_T_TH_horizontal_diffusion = T_TH_horizontal_diffusion*ATL_sig2_mask
atl_T_SLT_horizontal_diffusion = T_SLT_horizontal_diffusion*ATL_sig2_mask

atl_T_forcS = T_forcS*ATL_sig2_mask


so_T_TH_vertical_advection = T_TH_vertical_advection*so_basin_mask_C*SO_sig2_mask
so_T_SLT_vertical_advection = T_SLT_vertical_advection*so_basin_mask_C*SO_sig2_mask
so_T_TH_vertical_diffusion = T_TH_vertical_diffusion*so_basin_mask_C*SO_sig2_mask
so_T_SLT_vertical_diffusion = T_SLT_vertical_diffusion*so_basin_mask_C*SO_sig2_mask

so_T_TH_horizontal_advection = T_TH_horizontal_advection*so_basin_mask_C*SO_sig2_mask
so_T_SLT_horizontal_advection = T_SLT_horizontal_advection*so_basin_mask_C*SO_sig2_mask
so_T_TH_horizontal_diffusion = T_TH_horizontal_diffusion*so_basin_mask_C*SO_sig2_mask
so_T_SLT_horizontal_diffusion = T_SLT_horizontal_diffusion*so_basin_mask_C*SO_sig2_mask

so_T_forcS = T_forcS*so_basin_mask_C*SO_sig2_mask
so_T_forcS_nointerp = T_forcS_nointerp*so_basin_mask_C*SO_sig2_mask




In [29]:
save = True

# THETA / SALT sums
ip_sig2_total_saltconvergence = (ip_T_SLT_vertical_advection
                                 + ip_T_SLT_vertical_diffusion 
                                 + ip_T_SLT_horizontal_diffusion
                                 + ip_T_SLT_horizontal_advection
                                )
ip_sig2_total_thetaconvergence = (ip_T_TH_vertical_diffusion 
                                  + ip_T_TH_vertical_advection 
                                  + ip_T_TH_horizontal_advection 
                                  + ip_T_TH_horizontal_diffusion 
                                 )
 

atl_sig2_total_saltconvergence = (atl_T_SLT_vertical_advection 
                                  + atl_T_SLT_vertical_diffusion 
                                  + atl_T_SLT_horizontal_diffusion
                                  + atl_T_SLT_horizontal_advection
                                 )
atl_sig2_total_thetaconvergence = (atl_T_TH_vertical_diffusion 
                                   + atl_T_TH_vertical_advection 
                                   + atl_T_TH_horizontal_advection 
                                   + atl_T_TH_horizontal_diffusion 
                                  )


so_sig2_total_saltconvergence = (so_T_SLT_vertical_diffusion 
                                 + so_T_SLT_vertical_advection 
                                 + so_T_SLT_horizontal_diffusion                               
                                 + so_T_SLT_horizontal_advection
                                )
so_sig2_total_thetaconvergence = (so_T_TH_vertical_advection 
                                  + so_T_TH_vertical_diffusion 
                                  + so_T_TH_horizontal_advection 
                                  + so_T_TH_horizontal_diffusion 
                                 )

suffix = "_high_res"
if save:
    ip_sig2_total_thetaconvergence.to_netcdf(f"ip_sig2_total_thetaconvergence_FINAL{suffix}.nc")
    ip_sig2_total_saltconvergence.to_netcdf(f"ip_sig2_total_saltconvergence_FINAL{suffix}.nc")

    atl_sig2_total_thetaconvergence.to_netcdf(f"atl_sig2_total_thetaconvergence_FINAL{suffix}.nc")
    atl_sig2_total_saltconvergence.to_netcdf(f"atl_sig2_total_saltconvergence_FINAL{suffix}.nc")

    so_sig2_total_thetaconvergence.to_netcdf(f"so_sig2_total_thetaconvergence_FINAL{suffix}.nc")
    so_sig2_total_saltconvergence.to_netcdf(f"so_sig2_total_saltconvergence_FINAL{suffix}.nc")


